> **Binary** **classification** **model** : AntsBees  

> **Coding** **Stye**: TypeB

> **Section**: Model Class


In [ ]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import os
import torch
import torch.utils.data as data
from pathlib import Path


In [ ]:
data_dir = '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data'
root_dir= '/content/gdrive/My Drive/Colab Notebooks/AntsBees/data/hymenoptera_data'

In [ ]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/AntsBees')

# ***DataSet Class***

In [ ]:
from util.ImageTransform import ImageTransform
from dsets.dsets import make_path_list
from dsets.dsets import MakeBalancedDataset
from torch.utils.data import DataLoader


train_list = make_path_list(phase='train',root_dir=root_dir)
val_list = make_path_list(phase='val',root_dir=root_dir)

file_list={'train':train_list,'val':val_list}

SIZE = 224
# 標準化する際の各RGBの平均値
MEAN = (0.485, 0.456, 0.406) # ImageNetデータセットの平均値を使用
# 標準化する際の各RGBの標準偏差
STD = (0.229, 0.224, 0.225)  # ImageNetデータセットの標準偏差を使用


# 画像のサイズ、平均値、標準偏差の定数値
size, mean, std = SIZE, MEAN, STD

# MakeDatasetで前処理後の訓練データと正解ラベルを取得
train_dataset = MakeBalancedDataset(
    file_list=file_list, # 訓練データのファイルパス
    ratio_int=True,
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='train',records=300)
# MakeDatasetで前処理後の検証データと正解ラベルを取得
val_dataset = MakeBalancedDataset(
    file_list=file_list, # 検証データのファイルパス
    ratio_int=True,
    transform=ImageTransform(size, mean, std), # 前処理後のデータ
    phase='val',records=200)


# ミニバッチのサイズを指定
batch_size = 32

# 訓練用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 検証用のデータローダー:(バッチサイズ, 3, 224, 224)を生成
val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# ***Model Class***

In [ ]:
# 使用可能なデバイス(CPUまたはGPU）を取得する
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
'''
7. 学習済みのVGG16モデルをロード
'''
from torchvision import models
import torch.nn as nn

# ImageNetで事前トレーニングされたVGG16モデルを取得
model = models.vgg16(pretrained=True)

# VGG16の出力層のユニット数を2にする
model.classifier[6] = nn.Linear(
    in_features=4096, # 入力サイズはデフォルトの4096
    out_features=2)   # 出力はデフォルトの1000から2に変更


model = model.to(device)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

#model

In [ ]:
batch_iterator = iter(train_dl)
batch_tup = next(batch_iterator)


input_t, label_t = batch_tup

input_g=input_t.to(device)
label_g=label_t.to(device)
model = model.to(device)

outputs = model(input_g)

pred_label= outputs.argmax(dim=-1)
print(pred_label)
pred_label_list = pred_label.tolist()

print(pred_label_list)

print(outputs)
print(outputs.argmax(dim=-1).tolist())
print(label_t.tolist())
from sklearn.metrics import accuracy_score
print(accuracy_score(label_t.tolist(),pred_label.tolist()))



tensor([0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 1], device='cuda:0')
[0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1]
tensor([[ 0.3949, -0.4330],
        [-0.3094,  0.0589],
        [-0.5823, -0.6481],
        [-0.3344,  0.4486],
        [ 0.2961, -0.5022],
        [ 0.7091,  0.0515],
        [ 0.3827,  0.8520],
        [ 0.6909,  0.0019],
        [ 0.4416,  1.1244],
        [ 1.3374,  1.2046],
        [ 0.6885,  0.3449],
        [-0.2440, -1.5954],
        [ 0.3650,  0.1256],
        [ 1.1777, -0.1648],
        [ 0.8176, -0.1694],
        [-0.2314, -0.3727],
        [-0.0218, -0.1078],
        [ 0.5638, -0.3678],
        [ 0.4613, -0.3329],
        [-0.2402,  1.4619],
        [ 0.0152, -0.5131],
        [ 0.3267, -0.0840],
        [ 0.4424, -0.4133],
        [ 0.6422,  0.8668],
        [ 0.6817,  0.1885],
        [-0.2106, -0.5270],
        [ 0.1739,  0.2873],
        [ 0.5411,

In [ ]:
softmax = nn.Softmax(dim=1)
prob_g=softmax(outputs)
print(prob_g[:,1])

tensor([0.3041, 0.5910, 0.4836, 0.6863, 0.3104, 0.3413, 0.6152, 0.3343, 0.6644,
        0.4669, 0.4149, 0.2056, 0.4404, 0.2071, 0.2715, 0.4647, 0.4785, 0.2826,
        0.3113, 0.8458, 0.3709, 0.3987, 0.2982, 0.5559, 0.3791, 0.4216, 0.5283,
        0.2067, 0.7007, 0.4633, 0.5190, 0.7484], device='cuda:0',
       grad_fn=<SelectBackward0>)


In [ ]:
loss_func = nn.CrossEntropyLoss(reduction='none')
loss_g = loss_func(outputs,label_g)
print(loss_g)

tensor([1.1904, 0.8941, 0.7266, 0.3764, 0.3717, 0.4175, 0.9551, 0.4068, 1.0917,
        0.6290, 0.8796, 1.5817, 0.8200, 0.2321, 1.3038, 0.7663, 0.7371, 1.2637,
        1.1671, 0.1675, 0.4635, 0.5087, 1.2098, 0.5872, 0.9699, 0.5474, 0.7514,
        0.2315, 1.2065, 0.7693, 0.7319, 0.2898], device='cuda:0',
       grad_fn=<NllLossBackward0>)


In [ ]:
print(outputs)

tensor([[ 0.3949, -0.4330],
        [-0.3094,  0.0589],
        [-0.5823, -0.6481],
        [-0.3344,  0.4486],
        [ 0.2961, -0.5022],
        [ 0.7091,  0.0515],
        [ 0.3827,  0.8520],
        [ 0.6909,  0.0019],
        [ 0.4416,  1.1244],
        [ 1.3374,  1.2046],
        [ 0.6885,  0.3449],
        [-0.2440, -1.5954],
        [ 0.3650,  0.1256],
        [ 1.1777, -0.1648],
        [ 0.8176, -0.1694],
        [-0.2314, -0.3727],
        [-0.0218, -0.1078],
        [ 0.5638, -0.3678],
        [ 0.4613, -0.3329],
        [-0.2402,  1.4619],
        [ 0.0152, -0.5131],
        [ 0.3267, -0.0840],
        [ 0.4424, -0.4133],
        [ 0.6422,  0.8668],
        [ 0.6817,  0.1885],
        [-0.2106, -0.5270],
        [ 0.1739,  0.2873],
        [ 0.5411, -0.8041],
        [ 0.7096,  1.5605],
        [ 0.0685, -0.0785],
        [ 0.1729,  0.2489],
        [-0.2654,  0.8248]], device='cuda:0', grad_fn=<AddmmBackward0>)


#END